In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
cap = cv2.VideoCapture(0)
print(cap.isOpened())

In [ ]:
# close out of program by pressing q
while cap.isOpened():
    _, imageFrame = cap.read()
    hsvFrame = cv2.cvtColor(imageFrame, cv2.COLOR_BGR2HSV)
    blue_lower = np.array([100, 80, 2], np.uint8)
    blue_upper = np.array([120, 255, 255], np.uint8)
    blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)

    kernal = np.ones((5, 5), "uint8")
    
    blue_mask = cv2.dilate(blue_mask, kernal)
    res_blue = cv2.bitwise_and(imageFrame, imageFrame, mask = blue_mask)
    
    # Creating contour to track blue color
    contours, hierarchy = cv2.findContours(blue_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #print("area list",area_list)

    area_list = []
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        area_list.append(area)
        #print("area list", area_list)
        if(area > 1500):
            x, y, w, h = cv2.boundingRect(contour)
            imageFrame = cv2.rectangle(imageFrame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            #cv2.putText(imageFrame, "Blue Colour", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0))
            cv2.putText(imageFrame, f"Center Coordinate: {(x + w) / 2}, {(y + h) / 2}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0))
    cv2.imshow("Mask", blue_mask)
    cv2.imshow("Webcam", imageFrame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()